In [3]:
#Load the required libraries.

import gensim
import pandas as pd
import spacy
import nltk
import re
import pprint
import os
import sys

#from th_preprocessor.preprocess import preprocess
from gensim import corpora
from nltk import sent_tokenize
import spacy
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

#NLTK Stop words

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
nlp = spacy.load("en_core_web_sm")
nltk.download('stopwords')

#Extract the file from Excel_file: concepts_4_81.xlsx.

df = pd.read_excel ('concepts_4_81.xlsx') #change the path

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kimon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
#Print the column with name "Definition".

print(df.definition.unique())

["    The         abroad sector        consists of:        all institutions and individuals located outside the political frontiers of a country, except vehicles, ships, aircraft and space satellites operated by domestic entities and testing grounds acquired by such entities;         all international organisations (except business enterprises), including facilities and operations within the country's borders.    "
 '    An         acidification gas        or         acid gas        is a natural gas or gas mixture containing significant acid components, usually H         2        S or CO         2        .   '
 '    The         annual average growth rate        , abbreviated as         AAGR        and more accurately known as the         compound annual growth rate        , shows an         average        value for the annual rate of change over a period of time (typically several years) allowing for the compound effect of growth.       This rate facilitates comparisons of rates of cha

In [5]:
#Convert dataset to list and print a new dataset named as "data1".

from pprint import pprint

data = df.definition.values.tolist()                           
letters= re.sub("[^a-zA-Z]", " ",str(data))                                # Remove anything that is not a letter or a number
letters2 = re.sub("\'",  " ", str(letters))                                # Remove distracting single quotes
data1 = re.sub('\s+', ' ',str(letters2))                                   # Remove new line characters
pprint(data1)

(' The abroad sector consists of all institutions and individuals located '
 'outside the political frontiers of a country except vehicles ships aircraft '
 'and space satellites operated by domestic entities and testing grounds '
 'acquired by such entities all international organisations except business '
 'enterprises including facilities and operations within the country s borders '
 'An acidification gas or acid gas is a natural gas or gas mixture containing '
 'significant acid components usually H S or CO The annual average growth rate '
 'abbreviated as AAGR and more accurately known as the compound annual growth '
 'rate shows an average value for the annual rate of change over a period of '
 'time typically several years allowing for the compound effect of growth This '
 'rate facilitates comparisons of rates of change for periods of different '
 'lengths for example comparing annual five yearly and ten yearly rates of '
 'change This rate is calculated by taking the n th roo

 'is a specialised agency of the United Nations UN that acts as a coordinating '
 'body on public health issues Headquartered in Geneva Switzerland WHO s main '
 'task is to fight disease especially key infectious diseases and provide '
 'leadership on global health matters A weight in statistical terms is defined '
 'as a coefficient assigned to a number in a computation for example when '
 'determining an average to make the number s effect on the computation '
 'reflect its importance An illustration of weight and weighting is the '
 'calculation of the harmonised index of consumer prices HICP performed by '
 'Eurostat The HICP is composed of prices for a basket of items regularly '
 'purchased by consumers However some items such as food and newspapers are '
 'purchased more frequently than others such as car insurance or a television '
 'set To account for this the various items in the basket are assigned a '
 'weight to reflect the total consumer expenditure on these items even t

 'some of the taxes or other income that it receives according to its own '
 'policies The entity may however receive transfers from the central '
 'government that are tied to certain specified purposes A state government is '
 'able to appoint officers independently of external administrative control If '
 'a government entity operating in a state is entirely dependent on funds from '
 'central government and if central government also dictates the ways in which '
 'those funds are to be spent then the entity is an agency of central '
 'government The standard output of an agricultural product crop or livestock '
 'abbreviated as SO is the average monetary value of the agricultural output '
 'at farm gate price in euro per hectare or per head of livestock There is a '
 'regional SO coefficient for each product as an average value over a '
 'reference period years except for the SO coefficient calculated using the '
 'average of years The sum of all the SO per hectare of crop and per 

 'the minor differences between the PLIs and result in differences in ranking '
 'which are not statistically or economically significant Primary data '
 'constitute the most important inputs from among the plenitude of '
 'institutional administrative sample survey and or census based information '
 'used in compiling statistical aggregates Primary data is data observed or '
 'collected directly from first hand experience Published data and the data '
 'collected in the past or other parties is called secondary data The price '
 'level index abbreviated as PLI expresses the price level of a given country '
 'relative to another or relative to a group of countries like the European '
 'Union by dividing the Purchasing power parities PPPs by the current nominal '
 'exchange rate If the price level index of a country is higher than the '
 'country concerned is relatively expensive compared to the one to which it is '
 'compared for example the EU while if the price level index is lower t

 'fuels Biological digestion is intended to reduce the weight and to render '
 'inert any biologically active organic materials typically called stabilised '
 'residue Meat production refers to the slaughter in slaughterhouses and farms '
 'of animals whose carcass is declared fit for human consumption The '
 'definition applies to beef pork lamb and mutton Measure refers to the '
 'phenomenon or phenomena to be measured in a data set Technically a measure '
 'is a function that assigns a non negative real number to certain subsets of '
 'a dataset Means tested social benefits refer to benefits where entitlement '
 'is explicitly or implicitly conditional on the beneficiary s income wealth '
 'This covers cases where income wealth is used to determine only entitlement '
 'or both entitlement and amount The latter case refers to a benefit that is '
 'degressively linked to income wealth and reduced to zero as income rises the '
 'point at which the amount tapers to zero being the level 

 'resources in science and technology core Those people who have successfully '
 'completed a tertiary level education AND are employed in a S T occupation SE '
 'scientists and engineers Those people who work in ISCO groups Science and '
 'engineering professionals Health professionals Information and '
 'communications technology professionals HRSTU human resources in science and '
 'technology unemployed Those people who have successfully completed a '
 'tertiary level education AND are unemployed in any kind of job not only in a '
 'S T occupation NON HRSTU human resources NOT in science and technology '
 'unemployed Those people who have not fulfilled the HRST requirements AND are '
 'unemployed Household real income per capita is the adjusted gross disposable '
 'income of households in nominal terms divided by the total population source '
 'National Accounts and by the deflator price index of household final '
 'consumption expenditure The housing cost overburden rate is the pe

 'transactions of the EU Member States Intra EU on the other hand refers to '
 'all transactions occurring within the EU The term can have a different '
 'coverage depending on the perspective taken the EU as a whole a Member State '
 'a region or a city a port or an airport The exports of a particular Member '
 'State for instance can be split into two parts on the one hand to the '
 'countries outside the EU the rest of the world extra EU and on the other to '
 'the other Member States intra EU Departures from an airport can be extra EU '
 'or intra EU in the latter case they also include domestic flights within the '
 'Member State To complicate matters the EU has not always had the same '
 'composition due to consecutive enlargements For this reason both terms are '
 'sometimes further qualified as extra EU intra EU etc extra EU for instance '
 'refers to the rest of the world apart from the Member States that in the '
 'past made up EU External inflow is the volume of water flowin

 'provide Member States and the European institutions with the best possible '
 'scientific advice on questions related to the safety and the socio economic '
 'aspects of the use of chemicals The European Chemicals Agency comprises a '
 'Management Board responsible for adopting the financial planning work '
 'programme and annual reporting an Executive Director and a Secretariat a '
 'Member State Committee to resolve differences of opinion on draft decisions '
 'proposed by the Agency or Member States and make proposals for '
 'identification of substances of very high concern a Risk Assessment '
 'Committee to prepare opinions on evaluation on applications for '
 'authorisation on proposals for restrictions and on classification and '
 'labelling a Committee for Socio economic Analysis a Forum on enforcement '
 'matters to coordinate a network of Member States competent authorities '
 'responsible for enforcement a Board of Appeal to decide on appeals against '
 'decisions taken by

 'employment i e the total hours worked divided by the average annual hours '
 'worked in full time jobs in the country One annual work unit corresponds to '
 'the work performed by one person who is occupied on an agricultural holding '
 'on a full time basis The minimum working hours for work to be considered '
 'full time are defined in the national provisions governing contracts of '
 'employment If these do not indicate the number of annual hours then hours is '
 'to be taken as the minimum figure working days of eight hours each As the '
 'volume of agricultural labour is being calculated on the basis of full time '
 'equivalent jobs no person can therefore represent more than one AWU This '
 'constraint holds even if it is known that someone is working on agricultural '
 'activities for more than the number ofhours defining full time in the Member '
 'State concerned An avoidable cause of death also called avoidable death or '
 'avoidable mortality is a cause of death which coul

In [6]:
#Tokenize words and clean-up text.

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
data= df.definition.values.tolist()
data_words = list(sent_to_words(data))
print(data_words)

[['the', 'abroad', 'sector', 'consists', 'of', 'all', 'institutions', 'and', 'individuals', 'located', 'outside', 'the', 'political', 'frontiers', 'of', 'country', 'except', 'vehicles', 'ships', 'aircraft', 'and', 'space', 'satellites', 'operated', 'by', 'domestic', 'entities', 'and', 'testing', 'grounds', 'acquired', 'by', 'such', 'entities', 'all', 'international', 'organisations', 'except', 'business', 'enterprises', 'including', 'facilities', 'and', 'operations', 'within', 'the', 'country', 'borders'], ['an', 'acidification', 'gas', 'or', 'acid', 'gas', 'is', 'natural', 'gas', 'or', 'gas', 'mixture', 'containing', 'significant', 'acid', 'components', 'usually', 'or', 'co'], ['the', 'annual', 'average', 'growth', 'rate', 'abbreviated', 'as', 'aagr', 'and', 'more', 'accurately', 'known', 'as', 'the', 'compound', 'annual', 'growth', 'rate', 'shows', 'an', 'average', 'value', 'for', 'the', 'annual', 'rate', 'of', 'change', 'over', 'period', 'of', 'time', 'typically', 'several', 'years'

In [7]:
#Build the bigram and trigram models

bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

#Faster way to get a sentence clubbed as a trigram/bigram

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

#See trigram example

print(trigram_mod[bigram_mod[data_words[0]]])

['the', 'abroad', 'sector', 'consists', 'of', 'all', 'institutions', 'and', 'individuals', 'located', 'outside', 'the', 'political', 'frontiers', 'of', 'country', 'except', 'vehicles', 'ships', 'aircraft', 'and', 'space', 'satellites', 'operated', 'by', 'domestic', 'entities', 'and', 'testing', 'grounds', 'acquired', 'by', 'such', 'entities', 'all', 'international', 'organisations', 'except', 'business', 'enterprises', 'including', 'facilities', 'and', 'operations', 'within', 'the', 'country', 'borders']


In [8]:
#Define functions for stopwords, bigrams, trigrams and lemmatization

def remove_stopwords(texts):
    return [[word for word in gensim.utils.simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

#Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
#Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)


#Initialize spacy 'en' model, keeping only tagger component (for efficiency)
#python3 -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

#Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['abroad', 'sector', 'consist', 'institution', 'individual', 'locate', 'political', 'frontier', 'country', 'vehicle', 'ship', 'aircraft', 'space', 'satellite', 'operate', 'domestic', 'entity', 'testing', 'ground', 'acquire', 'entity', 'international', 'organisation', 'business', 'enterprise', 'include', 'facility', 'operation', 'country', 'border']]


In [9]:
#Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

#Create Corpus
texts = data_lemmatized

#Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

#View
print(corpus[:1])

#Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1)]]


[[('abroad', 1),
  ('acquire', 1),
  ('aircraft', 1),
  ('border', 1),
  ('business', 1),
  ('consist', 1),
  ('country', 2),
  ('domestic', 1),
  ('enterprise', 1),
  ('entity', 2),
  ('facility', 1),
  ('frontier', 1),
  ('ground', 1),
  ('include', 1),
  ('individual', 1),
  ('institution', 1),
  ('international', 1),
  ('locate', 1),
  ('operate', 1),
  ('operation', 1),
  ('organisation', 1),
  ('political', 1),
  ('satellite', 1),
  ('sector', 1),
  ('ship', 1),
  ('space', 1),
  ('testing', 1),
  ('vehicle', 1)]]

In [10]:
#Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [11]:
#Print the Keyword in the 10 topics
#The above LDA model is built with 20 different topics where each topic is a combination of keywords and each keyword contributes a certain weightage to the topic.

from pprint import pprint
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.131*"european" + 0.083*"union" + 0.079*"state" + 0.075*"member" + '
  '0.054*"euro" + 0.050*"area" + 0.045*"index" + 0.040*"economic" + '
  '0.034*"commission" + 0.017*"monetary"'),
 (1,
  '0.091*"bed" + 0.072*"housing" + 0.031*"manure" + 0.025*"exercise" + '
  '0.023*"soil" + 0.022*"science" + 0.022*"build" + 0.022*"type" + '
  '0.019*"normal" + 0.018*"house"'),
 (2,
  '0.210*"country" + 0.055*"trade" + 0.049*"resident" + 0.049*"international" '
  '+ 0.035*"january" + 0.031*"economy" + 0.028*"world" + 0.025*"currency" + '
  '0.025*"rate" + 0.018*"exchange"'),
 (3,
  '0.108*"region" + 0.074*"area" + 0.056*"table" + 0.036*"geographical" + '
  '0.032*"tree" + 0.031*"surface" + 0.029*"belgium" + 0.028*"refer" + '
  '0.026*"austria" + 0.024*"slovenia_si"'),
 (4,
  '0.055*"contribution" + 0.047*"water" + 0.044*"gas" + 0.040*"equivalent" + '
  '0.040*"waste" + 0.038*"emission" + 0.029*"weight" + 0.029*"natural" + '
  '0.028*"co" + 0.027*"include"'),
 (5,
  '0.091*"agricultural" + 0

In [12]:
#Visualize the topics

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
16    -0.146648  0.133357       1        1  13.856228
6     -0.262515 -0.027444       2        1  12.119628
11    -0.054622  0.203674       3        1   9.754915
13    -0.036455  0.211073       4        1   8.895787
10    -0.000687  0.136251       5        1   8.684997
12    -0.170312  0.008964       6        1   4.988779
0     -0.180985 -0.213632       7        1   4.420680
2     -0.119613 -0.033657       8        1   4.246977
19    -0.000165  0.050347       9        1   4.155553
8      0.099466  0.124408      10        1   4.093151
14     0.014190  0.055934      11        1   3.739718
5      0.117780 -0.022368      12        1   3.420251
9     -0.103245 -0.230989      13        1   3.116473
4      0.083614  0.005659      14        1   3.097281
15     0.035944 -0.080681      15        1   2.802641
7      0.173742  0.038819      16        1   2.528498
3      0.109306 -0.132347      17        1   1.900656
1      0.125204 -0.055258      18        1   1.867511
18     0.163962 -0.086980      19        1   1.169022
17     0.152039 -0.085130      20        1   1.141254, topic_info=           Term         Freq        Total Category  logprob  loglift
6       country  1610.000000  1610.000000  Default  30.0000  30.0000
153    european  1517.000000  1517.000000  Default  29.0000  29.0000
98     activity  1009.000000  1009.000000  Default  28.0000  28.0000
8    enterprise   786.000000   786.000000  Default  27.0000  27.0000
517       union   848.000000   848.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
300   available    14.934034   231.284119  Topic20  -4.5814   1.7330
919      direct    11.609026   188.693036  Topic20  -4.8332   1.6847
266         use    38.078465  1069.787804  Topic20  -3.6454   1.1375
338    economic    16.980094   723.854915  Topic20  -4.4530   0.7205
470     produce     7.859724   290.181970  Topic20  -5.2233   0.8643

[937 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
39        1  0.016960  abbreviate
39        2  0.290013  abbreviate
39        4  0.123807  abbreviate
39        6  0.178078  abbreviate
39        7  0.125503  abbreviate
...     ...       ...         ...
271      10  0.523731        work
272       2  0.984247      worker
630       8  0.990712       world
70        1  0.998633        year
71        1  0.976907      yearly

[1318 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 7, 12, 14, 11, 13, 1, 3, 20, 9, 15, 6, 10, 5, 16, 8, 4, 2, 19, 18])